In [53]:
import numpy as np
from scipy.optimize import minimize

In [54]:
N = 10
dt = 0.1
Lf = 2.67

ref_cte  = 0
ref_epsi = 0
ref_v = 60

x_start      = 0
y_start      = x_start       + N
psi_start    = y_start       + N
v_start      = psi_start     + N
cte_start    = v_start       + N
epsi_start   = cte_start     + N
delta_start  = epsi_start    + N
a_start      = delta_start   + N - 1

In [55]:
cte_weight = 200
epsi_weight = 200
v_weight = 10
actuator_cost_weight = 1
change_steer_cost_weight = 10000
change_acc_cost_weight = 1000

def objective(vars):
    cost = 0
    # cost due to difference from reference values
    for t in range(N):
        cost += cte_weight * (vars[cte_start + t] - ref_cte) ** 2
        cost += epsi_weight * (vars[epsi_start + t] - ref_epsi) ** 2
        cost += v_weight * (vars[v_start + t] - ref_v) ** 2
    
    # cost due to magnitude of actuators
    for t in range(N-1):
        cost += actuator_cost_weight * (vars[delta_start + t]) ** 2
        cost += actuator_cost_weight * (vars[a_start + t]) ** 2
         
    # cost due to magnitude of change in actuators
    for t in range(N-2):
        cost += change_steer_cost_weight * (vars[delta_start + t + 1] - vars[delta_start + t]) ** 2
        cost += change_acc_cost_weight * (vars[a_start + t + 1] - vars[a_start + t]) ** 2
     
    return np.log(cost)


In [72]:
def get_constraints():
#     coeffs = np.array([
#         (0, 0.0),
#         (0, 10.0),
#         (0, 20.0),
#         (0, 30.0)
#     ])
#     coeffs = np.array([
#         (0.0, 0),
#         (10.0, 0),
#         (20.0, 0),
#         (30.0, 0)
#     ])
    coeffs =  [ 1.375,  1.   ,  0.625,  2.5  ]
    constraints = [None] * (N * 6 + 1)
    constraints[x_start] = lambda vars: vars[x_start]
    constraints[y_start] = lambda vars: vars[y_start]
    constraints[psi_start] =  lambda vars: vars[psi_start]
#     constraints[v_start] =  lambda vars: vars[v_start]- 70
#     constraints[v_start+1] =  lambda vars: vars[v_start] - 70
    constraints[cte_start] =  lambda vars: vars[cte_start]
    constraints[epsi_start] =  lambda vars: vars[epsi_start]
    
    for t in range(1, N):
        # var0 at time (t) and var1 at time (t+1)
        # t=t for early binding
        def cons_x(vars, t=t):
            x0 = vars[x_start + t - 1]
            x1 = vars[x_start + t]
            v0 = vars[v_start + t - 1]
            psi0 = vars[psi_start + t - 1]
            return x1 - (x0 + v0 * np.cos(psi0) * dt)
        
        def cons_y(vars, t=t):
            y0 = vars[y_start + t - 1]
            y1 = vars[y_start + t]
            v0 = vars[v_start + t - 1]
            psi0 = vars[psi_start + t - 1]
            return y1 - (y0 + v0 * np.sin(psi0) * dt)
        
        def cons_psi(vars, t=t):
            v0 = vars[v_start + t - 1]
            psi0 = vars[psi_start + t - 1]
            psi1 = vars[psi_start + t]
            delta0 = vars[delta_start + t - 1]
            return psi1 - (psi0 - v0 * delta0 / Lf * dt)
        
        def cons_v(vars, t=t):
            v0 = vars[v_start + t - 1]
            v1 = vars[v_start + t]
            a0 = vars[a_start + t - 1]
            return v1 - (v0 + a0 * dt)
        
        def cons_cte(vars, t=t):
            x0 = vars[x_start + t - 1]
            y0 = vars[y_start + t - 1]
            v0 = vars[v_start + t - 1]
            f0 = coeffs[0] + coeffs[1] * x0 + coeffs[2] * x0 ** 2 + coeffs[3] * x0 ** 3
            cte1 = vars[cte_start + t]
            epsi0 = vars[epsi_start + t - 1]
            return cte1 - (f0 - y0 + (v0 * np.sin(epsi0) * dt))
        
        def cons_epsi(vars, t=t):
            x0 = vars[x_start + t - 1]
            psi0 = vars[psi_start + t - 1]
            v0 = vars[v_start + t - 1]
            delta0 = vars[delta_start + t - 1]
            psides0 = np.arctan(coeffs[1] + 2 * coeffs[2] * x0 + 3 * coeffs[3] * x0 ** 2)
            epsi1 = vars[epsi_start + t]
            return epsi1 - (psi0 - psides0 - v0 * delta0 / Lf * dt)
        
        constraints[x_start + t] =  cons_x
        constraints[y_start + t] = cons_y
        constraints[psi_start + t] = cons_psi
        constraints[v_start + t + 1] = cons_v
        constraints[cte_start + t] = cons_cte
        constraints[epsi_start + t] = cons_epsi
        
#     constraints.append(lambda vars: vars[v_start] - 20)
    return constraints


constraints = [{'type': 'eq', 'fun': func} for func in get_constraints() if func is not None]
# for i in constraints:
#     print(i['fun'](vars))

In [73]:
import sys
n_vars = N * 6 + (N - 1) * 2
n_contrains = N * 6
bounds = [None] * n_vars
vars = [0] * n_vars

for i in range(delta_start):
    bounds[i] = (np.finfo(np.float16).min, np.finfo(np.float16).max)
    bounds[i] = [-1000, 1000]

max_radians = 25 * np.pi / 180
for i in range(delta_start, a_start):
    bounds[i] = (-max_radians, max_radians)
    
max_acc_value = 1.0
for i in range(a_start, n_vars):
    bounds[i] = (-max_acc_value, max_acc_value)

speed = 70
# vars[v_start] = speed
bounds[v_start] = (speed, speed)

In [74]:
solution = minimize(objective, vars, bounds=bounds, constraints=constraints, options={'disp':True, 'maxiter':1000})
print(solution.success)
print(solution.message)

Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: 20.21176320141177
            Iterations: 188
            Function evaluations: 14872
            Gradient evaluations: 184
False
Positive directional derivative for linesearch


In [272]:
for i in range(N-1):
    print(solution.x[a_start+i])

-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
0.9999999999994412
1.0


solution.x

In [187]:
print(vars)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]


In [132]:
contrainsts

NameError: name 'contrainsts' is not defined

In [68]:
for i in range(N):
    print(solution.x[a_start+i], solution.x[delta_start+i])

0.9999999999999899 0.4255463520092819
0.9999999999999617 0.42753375280266614
0.9999999999999112 0.4303920812947667
0.9999999999998364 0.43322405909024947
0.9999999999963555 0.4353533365169461
0.9999999999888491 0.43633231299602454
0.9999999999807816 0.43633231299600805
0.999999999976781 0.43633231299602765
0.9999999999883126 0.43633231299606623


IndexError: index 78 is out of bounds for axis 0 with size 78

In [995]:
for i in get_constraints():
    print(i)

None
<function get_constraints.<locals>.cons_x at 0x00000213F8431AE8>
<function get_constraints.<locals>.cons_x at 0x00000213FEE11BF8>
<function get_constraints.<locals>.cons_x at 0x00000213FEF701E0>
<function get_constraints.<locals>.cons_x at 0x00000213FEF70EA0>
<function get_constraints.<locals>.cons_x at 0x00000213FEF70620>
<function get_constraints.<locals>.cons_x at 0x00000213F8453730>
<function get_constraints.<locals>.cons_x at 0x00000213FEEF9F28>
<function get_constraints.<locals>.cons_x at 0x00000213FEF34620>
<function get_constraints.<locals>.cons_x at 0x00000213FEF34268>
None
<function get_constraints.<locals>.cons_y at 0x00000213F8431C80>
<function get_constraints.<locals>.cons_y at 0x00000213FEE11268>
<function get_constraints.<locals>.cons_y at 0x00000213FEF70598>
<function get_constraints.<locals>.cons_y at 0x00000213FEF70378>
<function get_constraints.<locals>.cons_y at 0x00000213FEF70268>
<function get_constraints.<locals>.cons_y at 0x00000213F84539D8>
<function get_c

In [756]:
def cons_cte(vars, t=0):
    x0 = vars[x_start + t - 1]
    y0 = vars[y_start + t - 1]
    v0 = vars[v_start + t - 1]
    f0 = coeffs[0] + coeffs[1] * x0 + coeffs[2] * x0 ** 2 + coeffs[3] * x0 ** 3
    cte1 = vars[cte_start + t]
    epsi0 = vars[epsi_start + t - 1]
    return cte1 - (f0 - y0 + (v0 * np.sin(epsi0) * dt))

In [697]:
np.finfo(np.float32).max

3.4028235e+38

In [747]:
type(70 - vars[2])

numpy.float64

In [760]:
vars + 100

array([100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
       100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
       100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
       100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
       100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
       100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
       100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
       100.])

In [998]:
constraints

[{'fun': <function __main__.get_constraints.<locals>.cons_x>, 'type': 'eq'},
 {'fun': <function __main__.get_constraints.<locals>.cons_x>, 'type': 'eq'},
 {'fun': <function __main__.get_constraints.<locals>.cons_x>, 'type': 'eq'},
 {'fun': <function __main__.get_constraints.<locals>.cons_x>, 'type': 'eq'},
 {'fun': <function __main__.get_constraints.<locals>.cons_x>, 'type': 'eq'},
 {'fun': <function __main__.get_constraints.<locals>.cons_x>, 'type': 'eq'},
 {'fun': <function __main__.get_constraints.<locals>.cons_x>, 'type': 'eq'},
 {'fun': <function __main__.get_constraints.<locals>.cons_x>, 'type': 'eq'},
 {'fun': <function __main__.get_constraints.<locals>.cons_x>, 'type': 'eq'},
 {'fun': <function __main__.get_constraints.<locals>.cons_y>, 'type': 'eq'},
 {'fun': <function __main__.get_constraints.<locals>.cons_y>, 'type': 'eq'},
 {'fun': <function __main__.get_constraints.<locals>.cons_y>, 'type': 'eq'},
 {'fun': <function __main__.get_constraints.<locals>.cons_y>, 'type': 'eq'},

In [13]:
from scipy import interpolate

pts_x = [0, 20, 40, 60, 80]
pts_y = [0, 0, 6, 6, 6]
tck = interpolate.CubicSpline(pts_x, pts_y)


In [14]:
# interpolate.splev([6,2], tck)
tck.c


array([[-3.4375e-04, -3.4375e-04,  2.1875e-04,  2.1875e-04],
       [ 2.8125e-02,  7.5000e-03, -1.3125e-02,  0.0000e+00],
       [-4.2500e-01,  2.8750e-01,  1.7500e-01, -8.7500e-02],
       [ 0.0000e+00,  0.0000e+00,  6.0000e+00,  6.0000e+00]])

In [1]:
func = tck
interpolate.spltopp(func[0][1:-1],func[1],func[2])

NameError: name 'tck' is not defined

In [20]:
interpolate.PPoly.from_spline(tck).c

array([[-0.125, -0.125, -0.125, -0.125,  0.625,  0.625,  0.625,  0.625],
       [ 1.375,  1.375,  1.375,  1.375,  0.625,  4.375,  4.375,  4.375],
       [-0.25 , -0.25 , -0.25 , -0.25 ,  3.75 , 13.75 , 13.75 , 13.75 ],
       [ 1.   ,  1.   ,  1.   ,  1.   ,  5.   , 20.   , 20.   , 20.   ]])